In [ ]:
# This notebook is NOT executable. It is only provided as part of the documentation for the analysis.
# Please make your copy and take care of the paths to your data and tools.

In [ ]:
# mRNA pipeline on modENCODE data

In [ ]:
%%bash
mRNA-seq -i path/to/fastq -o path/to/output -v -j 20 --trim --fastqc dm6_ensembl96.yaml

In [ ]:
#deseq size factor
#rawcount/sf
#log2

In [ ]:
import numpy as np 
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
# read raw count table

In [ ]:
modEnc_rawCount = "path/to/output/featureCounts/counts.tsv"
genes_id = "path/to/grouping.gene_id.tsv"

In [ ]:
counts = pd.read_csv(modEnc_rawCount, sep = "\t")
counts.rename(columns={"Unnamed: 0": "gene_id"}, inplace = True)

In [ ]:
# this part has been done in R, not here!

In [ ]:
# %R #(Doesn't work in notebook, it needs to be fixed, for now the code just got copied from running R)
# #On snakepipes_RNAseq_environment_0.1
#R (on terminal)
# library(DESeq2)
# counts <- read.table(modEnc_rawCount)
# deseq2 <- DESeq2::estimateSizeFactorsForMatrix(counts[,-1], locfunc = stats::median)
# write.table(deseq2, "deseq2_factors.tsv", sep = "\t" )

In [ ]:
deseq_norm = counts.copy()
deseq2_sf = pd.read_csv("deseq2_factors.tsv", sep = "\t", header = None)
deseq2_sf = deseq2_sf.T
new_header = deseq2_sf.iloc[0] 
deseq2_sf = deseq2_sf[1:] 
deseq2_sf.columns = new_header 
deseq2_sf.reset_index(inplace=True,drop=True)
for col in deseq_norm.columns:
    if col is not 'gene_id':
        deseq_norm[col] = deseq_norm[col] + 1 # pseudo count
        deseq_norm[col] = deseq_norm[col]/float(deseq2_sf[col])


In [ ]:
#Average of replicates
avg_counts = deseq_norm.copy()

for time in ["0-2","2-4","10-12","12-14","16-18","18-20","20-22","22-24"]:
    reps = ["embryos_"+time+"_hr.x", "embryos_"+time+"_hr.y"]
    avg_counts[time] = avg_counts[reps].mean(axis=1)
    avg_counts.drop(reps, inplace= True, axis =1)
    for time in ["14-16","6-8","4-6","8-10"]:
        rep = "embryos_"+time+"_hr"
        avg_counts.rename(columns={rep: time}, inplace = True)
avg_counts = avg_counts[["gene_id","0-2","2-4","4-6","6-8","8-10","10-12","12-14","14-16","16-18","18-20","20-22","22-24"]]

In [ ]:
#log2
log2_counts = avg_counts.copy()
for col in ["0-2","2-4","4-6","6-8","8-10","10-12","12-14","14-16","16-18","18-20","20-22","22-24"]:
    log2_counts[col] = (log2_counts[col]).astype(float)
    log2_counts[col] = np.log2(log2_counts[col]) # if value==o then log2(1)=0

In [ ]:
log2_counts.to_csv("log2_deseq2_normalised_counts.tsv", sep = "\t", index=False)

In [ ]:
dafnes_genes = pd.read_csv(genes_id, sep = "\t", usecols=["gene_id","fig3.group"])
mapped = pd.merge(log2_counts, dafnes_genes, how='inner', on = 'gene_id') 

In [ ]:
# sort by 'fig3.group' 
mapped.sort_values(by=['fig3.group'],inplace=True)
mapped.reset_index(inplace=True,drop=True)